In [79]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime

# Tasks

### Your assigned tasks components
● Create a working ETL using the sample file provided from the old system
(historical_orders_2B9S_LTD.csv)

○ The ETL must read the data, perform any scrubbing and land it in the data model

● Update the Dimensional Model as applicable and provide the ERD

### New Requirements Additional

● The sample data is supposed to be reflective of the data in the old system but there will be data coming in the new system, which has a different data structure.

● The old system only stored addresses at the time of order but the new system is going to have customer address versioning

● The new sales system will also have order statuses

● Provide a sample reports for the following use cases:

○ The ability to look at the total number of orders shipped to a specific state in a given year

○ The ability to look up all customers current addresses

○ Summary reports that will show all orders and their order status

○ A way to see what customers did not have order in certain years

■ Marketing is really interested in evaluating customer order trends

○ Quarterly Sales reports at the product level as well a quarterly report at the Data Model customer level
    

# Data

In [31]:
# Import data and create data frame
orders_df = pd.read_csv('data/historical_orders_tumbleweed_capital.csv')
orders_df.head()

,order_id,first_name,last_name,date_of_birth,phone,email,street,city,postal_code,state,order_date,payment_type,product_name,price,quantity,order_total
0,1,Sage,Wieser,9/16/98,605-414-2147,sage_wieser@cox.net,5 Boston Ave #88,Sioux Falls,57105,SD,10/1/16,VISA,Strongtam,18.99,4,75.96
1,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,11/29/16,VISA,Runhome,2.99,3,158.93
2,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,11/29/16,VISA,Runhome,71.99,2,158.93
3,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,11/29/16,VISA,Vilaity,2.99,2,158.93
4,4,Solange,Shinko,1/12/79,504-979-9175,solange@shinko.com,84 Bloomfield Ave,Spartanburg,29301,SC,6/9/13,ACH,Solo-In,72.99,3,218.97


In [34]:
# Check info and nulls
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       224 non-null    int64         
 1   first_name     224 non-null    object        
 2   last_name      224 non-null    object        
 3   date_of_birth  224 non-null    object        
 4   phone          224 non-null    object        
 5   email          224 non-null    object        
 6   street         224 non-null    object        
 7   city           224 non-null    object        
 8   postal_code    224 non-null    int64         
 9   state          224 non-null    object        
 10  order_date     224 non-null    datetime64[ns]
 11  payment_type   224 non-null    object        
 12  product_name   224 non-null    object        
 13  price          224 non-null    float64       
 14  quantity       224 non-null    int64         
 15  order_total    224 non-

In [10]:
# Check for duplicates
orders_df.duplicated().sum()

0

In [35]:
# Change order_date to date time
orders_df['order_date'] = pd.to_datetime(orders_df['order_date'], format= '%m/%d/%y')
orders_df.head()

,order_id,first_name,last_name,date_of_birth,phone,email,street,city,postal_code,state,order_date,payment_type,product_name,price,quantity,order_total
0,1,Sage,Wieser,9/16/98,605-414-2147,sage_wieser@cox.net,5 Boston Ave #88,Sioux Falls,57105,SD,2016-10-01,VISA,Strongtam,18.99,4,75.96
1,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,2016-11-29,VISA,Runhome,2.99,3,158.93
2,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,2016-11-29,VISA,Runhome,71.99,2,158.93
3,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,2016-11-29,VISA,Vilaity,2.99,2,158.93
4,4,Solange,Shinko,1/12/79,504-979-9175,solange@shinko.com,84 Bloomfield Ave,Spartanburg,29301,SC,2013-06-09,ACH,Solo-In,72.99,3,218.97


In [38]:
# Get first and last orders dates
orders_df['order_date'].min(), orders_df['order_date'].max()

(Timestamp('2013-02-14 00:00:00'), Timestamp('2017-02-13 00:00:00'))

In [41]:
# Create quarters
orders_df['quarter'] = orders_df['order_date'].dt.to_period('Q')
orders_df.head()

,order_id,first_name,last_name,date_of_birth,phone,email,street,city,postal_code,state,order_date,payment_type,product_name,price,quantity,order_total,quarter
0,1,Sage,Wieser,9/16/98,605-414-2147,sage_wieser@cox.net,5 Boston Ave #88,Sioux Falls,57105,SD,2016-10-01,VISA,Strongtam,18.99,4,75.96,2016Q4
1,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,2016-11-29,VISA,Runhome,2.99,3,158.93,2016Q4
2,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,2016-11-29,VISA,Runhome,71.99,2,158.93,2016Q4
3,3,Bette,Nicka,12/28/78,610-545-3615,bette_nicka@cox.net,6 S 33rd St,Aston,19014,PA,2016-11-29,VISA,Vilaity,2.99,2,158.93,2016Q4
4,4,Solange,Shinko,1/12/79,504-979-9175,solange@shinko.com,84 Bloomfield Ave,Spartanburg,29301,SC,2013-06-09,ACH,Solo-In,72.99,3,218.97,2013Q2


In [66]:
# Cleaning new data
class Clean_Data:
    
    def first_name(self, first):
        f = first.lower()
        f = f.capitalize()
        return f
        
    def last_name(self, last):
        l = last.lower()
        l = l.capitalize()
        return l   
    
    def order_date(self, date):
        d = pd.to_datetime(date, format= '%m/%d/%y')
        return d
    
    
    def payment_type(self, payment):
        p = payment.upper()
        return p
    
     def product_name(self, product):
        p = product.lower()
        p = p.capitalize()
        return p

In [82]:
  def order_date(date):
        dt_obj = datetime.strptime(date, "%d%m%y")
        dt_formatted = dt_obj.date()
        return dt_formatted

In [83]:
order_date('123465')

ValueError: unconverted data remains: 5

# Visuals

In [56]:
sns.histplot(data= orders_df, x= products, y= quarters)

ValueError: Length of DataFrameGroupBy vectors must match length of `data` when both are used, but `data` has length 224 and the vector passed to `x` has length 24.